In [2]:
import golz3
from frames import Frames
import z3


In [ ]:

n = 10 
period = 1
solver = z3.Solver()
state = golz3.make_life(solver, grid_size=n, time_steps=period + 1)
golz3.constrain_life_boundary(solver, state)
for i in range(n):
    for j in range(n):
        solver.add(state[0][i][j] == state[-1][i][j])

# At least one cell is alive.
solver.add(z3.Or([state[0][i][j] == 1 for i in range(n) for j in range(n)]))

# At least one cell differs between steps.
# for t in range(period):
#     solver.add(z3.Or([state[t][i][j] != state[t + 1][i][j] for i in range(n) for j in range(n)]))

if solver.check() == z3.sat:
    model = solver.model()
    golz3.print_model(model, state)
    Frames(golz3.model_to_python(model, state)[:-1]).save(f"{period}-periodic-{n}x{n}.golz3")
else:
    print("unsat")

In [72]:
def find_glider(glider_n, dx, dy, period):
    n = glider_n + max(dx, dy)
    solver = z3.Solver()
    state = golz3.make_life(solver, grid_size=n, time_steps=period + 1)
    golz3.constrain_life_boundary(solver, state)
    for i in range(glider_n):
        for j in range(glider_n):
            solver.add(state[0][i][j] == state[-1][i + dx][j + dy])
    for i in range(n):
        for j in range(n):
            if i >= glider_n or j >= glider_n:
                solver.add(state[0][i][j] == 0)
            if i < dx or j < dy:
                solver.add(state[-1][i][j] == 0)

    # At least one cell is alive.
    solver.add(z3.Or([state[0][i][j] == 1 for i in range(glider_n) for j in range(glider_n)]))

    # At least one cell differs between steps.
    # for t in range(period):
    #     solver.add(z3.Or([state[t][i][j] != state[t + 1][i][j] for i in range(n) for j in range(n)]))

    if solver.check() == z3.sat:
        model = solver.model()
        golz3.print_model(model, state)
        Frames(golz3.model_to_python(model, state)).save(f"{period}-glider-{glider_n}x{glider_n}.golz3")
        return True
    else:
        # print("unsat")
        return False

dx = 2
dy = 0
for glider_n in range(3, 9):
    for period in range(2, 6):
        print(f"trying {glider_n}x{glider_n} glider with dx={dx}, dy={dy}, period={period}")
        if find_glider(glider_n, dx, dy, period):
            break

trying 3x3 glider with dx=2, dy=0, period=2
trying 3x3 glider with dx=2, dy=0, period=3
trying 3x3 glider with dx=2, dy=0, period=4
trying 3x3 glider with dx=2, dy=0, period=5
trying 4x4 glider with dx=2, dy=0, period=2
trying 4x4 glider with dx=2, dy=0, period=3
trying 4x4 glider with dx=2, dy=0, period=4
trying 4x4 glider with dx=2, dy=0, period=5
trying 5x5 glider with dx=2, dy=0, period=2
trying 5x5 glider with dx=2, dy=0, period=3
trying 5x5 glider with dx=2, dy=0, period=4
t = 0
0 0 1 1 0 0 0 
0 1 1 1 0 0 0 
0 1 1 0 1 0 0 
0 0 1 1 1 0 0 
0 0 0 1 0 0 0 
0 0 0 0 0 0 0 
0 0 0 0 0 0 0 

t = 1
0 1 0 1 0 0 0 
0 0 0 0 1 0 0 
0 0 0 0 1 0 0 
0 1 0 0 1 0 0 
0 0 1 1 1 0 0 
0 0 0 0 0 0 0 
0 0 0 0 0 0 0 

t = 2
0 0 0 0 0 0 0 
0 0 0 1 1 0 0 
0 0 0 1 1 1 0 
0 0 1 0 1 1 0 
0 0 1 1 1 0 0 
0 0 0 1 0 0 0 
0 0 0 0 0 0 0 

t = 3
0 0 0 0 0 0 0 
0 0 0 1 0 1 0 
0 0 1 0 0 0 0 
0 0 1 0 0 0 0 
0 0 1 0 0 1 0 
0 0 1 1 1 0 0 
0 0 0 0 0 0 0 

t = 4
0 0 0 0 0 0 0 
0 0 0 0 0 0 0 
0 0 1 1 0 0 0 
0 1 1 1 0 0 0 
0 

In [9]:
n = 6
solver = z3.Solver()
state = golz3.make_life(solver, grid_size=n, time_steps=5)
pat = [(1, 1), (1, 4), (3, 1), (4, 2), (4, 3), (3, 4)]

for t in range(len(state)):
    for i, j in pat:
        solver.add(state[t][i][j] == 1)

# At least one cell differs between steps.
for t in range(len(state) - 1):
    solver.add(z3.Or([state[t][i][j] != state[t + 1][i][j] for i in range(n) for j in range(n)]))

if solver.check() == z3.sat:
    model = solver.model()
    golz3.print_model(model, state)
    Frames(golz3.model_to_python(model, state)).save(f"persist-smiley.golz3")
else:
    print("unsat")

t = 0
0 1 0 0 1 0 
0 1 0 0 1 0 
0 1 1 0 1 0 
0 1 0 1 1 0 
0 0 1 1 0 0 
0 0 0 0 0 1 

t = 1
0 0 0 0 0 0 
1 1 0 0 1 1 
1 1 0 0 1 1 
0 1 0 0 1 0 
0 0 1 1 0 0 
0 0 0 0 0 0 

t = 2
0 0 0 0 0 0 
1 1 0 0 1 1 
0 0 1 1 0 0 
1 1 0 0 1 1 
0 0 1 1 0 0 
0 0 0 0 0 0 

t = 3
0 0 0 0 0 0 
0 1 1 1 1 0 
0 0 1 1 0 0 
0 1 0 0 1 0 
0 1 1 1 1 0 
0 0 0 0 0 0 

t = 4
0 0 1 1 0 0 
0 1 0 0 1 0 
0 0 0 0 0 0 
0 1 0 0 1 0 
0 1 1 1 1 0 
0 0 1 1 0 0 

